In [1]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import numpy as np

from simulation import run_simulation
from graph import const_kregular

init_notebook_mode(connected=True)

In [2]:
#Define number of transmissions rate to space from 1-10
NUM_TRANS_RATES = 10
#Define number of times to run simulation per transmission rate
TRANS_RUN_COUNT = 10

#Amount of time to allocate
TIME_TO_ALLOC = 100
#Size of graph
NUM_VERTICES = 100
#Degree of vertices
VTX_DEGREE = 4

trans_rates = np.linspace(0, 1, NUM_TRANS_RATES + 1)[1:]

In [3]:
#Run simulations and store results
simul_utils = {}
seq_utils = {}
for t_rate in trans_rates:
    simul_utils[t_rate] = []
    seq_utils[t_rate] = []
    for i in range(TRANS_RUN_COUNT):
        G = const_kregular(VTX_DEGREE, NUM_VERTICES,
                                 t_rate, TIME_TO_ALLOC)
        _, simul_tot = run_simulation(G, simul=True)
        
        G = const_kregular(VTX_DEGREE, NUM_VERTICES,
                                 t_rate, TIME_TO_ALLOC)
        _, seq_tot= run_simulation(G, simul=False)
        
        #Get proportion of social optimal in each iteration
        simul_prop = [ sum(ut) / NUM_VERTICES for ut in simul_tot ]
        seq_prop = [ sum(ut) / NUM_VERTICES for ut in seq_tot ]
        simul_utils[t_rate].append(simul_prop)
        seq_utils[t_rate].append(seq_prop)

In [4]:
_green = 200
_blue = 200
red_vals = np.linspace(0, 225, NUM_TRANS_RATES)

def plot_utils(utils):
    fig = go.Figure()

    t_idx = 0
    for t_rate, trate_utils in utils.items():
        marker_color = 'rgb({0},{1},{2})'.format(
            red_vals[t_idx], _green, _blue)
        for util_seq in trate_utils:
            x_vals = list(range(len(util_seq)))
            fig.add_trace(go.Scatter(x=x_vals,
                y=util_seq, mode='markers', hoverinfo='skip',
                marker=dict(color=marker_color)))
        t_idx += 1

    fig.layout.update(showlegend=False)
    plot_title = "Percent social optimal vs Iteration Num"
    fig.layout.update(title=plot_title)

    iplot(fig)

In [5]:
plot_utils(simul_utils)

In [6]:
plot_utils(seq_utils)